In [1]:
import pandas as pd

df = pd.read_csv("../data/cancer_prepared.csv")

In [2]:
df.head()

,Age_bin_1,Age_bin_2,Age_bin_3,Genetic_Risk_bin_1,Genetic_Risk_bin_2,Genetic_Risk_bin_3,Air_Pollution_bin_1,Air_Pollution_bin_2,Air_Pollution_bin_3,Alcohol_Use_bin_1,...,Cancer_Type=Leukemia,Cancer_Type=Liver,Cancer_Type=Lung,Cancer_Type=Prostate,Cancer_Type=Skin,Cancer_Stage=Stage 0,Cancer_Stage=Stage I,Cancer_Stage=Stage II,Cancer_Stage=Stage III,Cancer_Stage=Stage IV
0,False,False,True,False,True,False,True,False,False,False,...,False,False,True,False,False,False,False,False,True,False
1,True,False,False,True,False,False,False,True,False,False,...,True,False,False,False,False,True,False,False,False,False
2,False,False,True,False,False,True,False,False,True,True,...,False,False,False,False,False,False,False,True,False,False
3,True,False,False,True,False,False,True,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4,True,False,False,False,True,False,True,False,False,True,...,False,False,False,False,True,False,False,False,True,False


In [3]:
from fca.api_models import Context
from src.utils import create_context_from_dataframe, show_rules, show_concepts, get_rules_for_rounds, \
    print_top_conf1_rules_by_support

In [4]:
num_of_rows = 2000
df_small = df.head(num_of_rows)

In [5]:
objects, attributes, relation = create_context_from_dataframe(df)
context = Context(objects, attributes, relation)

objects, attributes, relation = create_context_from_dataframe(df_small)
context_small = Context(objects, attributes, relation)

In [6]:
concepts = context_small.get_concepts()

In [7]:
len(concepts)

441269

In [8]:
show_concepts(concepts, 0.15, 2, num_of_rows)

1: ['Genetic_Risk_bin_1', 'Target_Severity_Score_bin_1'] - Support : 0.185
2: ['Genetic_Risk_bin_3', 'Target_Severity_Score_bin_3'] - Support : 0.1825
3: ['Air_Pollution_bin_1', 'Target_Severity_Score_bin_1'] - Support : 0.1845
4: ['Air_Pollution_bin_3', 'Target_Severity_Score_bin_3'] - Support : 0.154
5: ['Alcohol_Use_bin_1', 'Target_Severity_Score_bin_1'] - Support : 0.1675
6: ['Alcohol_Use_bin_3', 'Target_Severity_Score_bin_3'] - Support : 0.1575
7: ['Smoking_bin_1', 'Target_Severity_Score_bin_1'] - Support : 0.188
8: ['Smoking_bin_3', 'Target_Severity_Score_bin_3'] - Support : 0.18
9: ['Obesity_Level_bin_1', 'Target_Severity_Score_bin_1'] - Support : 0.1635


In [9]:
rules = list(context.get_association_rules(min_support=0.15, min_confidence=0.5))

In [10]:
show_rules(rules)

1: Target_Severity_Score_bin_1 -> Air_Pollution_bin_1
Support: 0.1689 - Confidence: 0.5025 - Lift: 1.4662
2: Genetic_Risk_bin_1 -> Target_Severity_Score_bin_1
Support: 0.1846 - Confidence: 0.5524 - Lift: 1.6431
3: Genetic_Risk_bin_3 -> Target_Severity_Score_bin_3
Support: 0.1783 - Confidence: 0.5486 - Lift: 1.6537
4: Smoking_bin_1 -> Target_Severity_Score_bin_1
Support: 0.1859 - Confidence: 0.5525 - Lift: 1.6436
5: Smoking_bin_3 -> Target_Severity_Score_bin_3
Support: 0.1840 - Confidence: 0.5530 - Lift: 1.6672


ESS Dataset

In [11]:
df = pd.read_csv("../data/essround1.csv")

In [12]:
df.head()

,aesfdrk_1_2,aesfdrk_2_2,clsprty_1_3,clsprty_2_3,clsprty_3_3,freehms_1_3,freehms_2_3,freehms_3_3,gincdif_1_3,gincdif_2_3,...,trstplc_3_3,trstplt_1_3,trstplt_2_3,trstplt_3_3,trstprl_1_3,trstprl_2_3,trstprl_3_3,trstun_1_3,trstun_2_3,trstun_3_3
0,False,False,True,False,False,False,False,False,False,False,...,True,False,False,True,False,False,True,False,False,False
1,True,False,True,False,False,False,True,False,False,False,...,True,False,True,False,False,False,True,False,True,False
2,True,False,True,False,False,False,True,False,False,False,...,True,False,True,False,False,False,True,False,False,True
3,False,True,True,False,False,True,False,False,True,False,...,True,False,False,True,False,False,True,False,False,True
4,False,True,True,False,False,True,False,False,True,False,...,True,False,False,True,False,False,True,False,True,False


In [13]:
num_of_rows = 20
df_small = df.head(num_of_rows)

In [14]:
objects, attributes, relation = create_context_from_dataframe(df)
context = Context(objects, attributes, relation)

objects, attributes, relation = create_context_from_dataframe(df_small)
context_small = Context(objects, attributes, relation)

In [15]:
rules = list(context.get_association_rules(min_support=0.7, min_confidence=0.9))

In [16]:
show_rules(rules)

1: aesfdrk_2_2 -> clsprty_1_3
Support: 0.7651 - Confidence: 0.9791 - Lift: 1.0034
2: freehms_1_3 -> clsprty_1_3
Support: 0.7667 - Confidence: 0.9792 - Lift: 1.0035
3: happy_3_3 -> clsprty_1_3
Support: 0.8236 - Confidence: 0.9793 - Lift: 1.0036
4: health_1_3 -> clsprty_1_3
Support: 0.7662 - Confidence: 0.9798 - Lift: 1.0041
5: impcntr_1_3 -> clsprty_1_3
Support: 0.8283 - Confidence: 0.9782 - Lift: 1.0025
6: impenv_1_3 -> clsprty_1_3
Support: 0.7056 - Confidence: 0.9788 - Lift: 1.0031
7: ipeqopt_1_3 -> clsprty_1_3
Support: 0.7346 - Confidence: 0.9776 - Lift: 1.0018
8: iplylfr_1_3 -> clsprty_1_3
Support: 0.8420 - Confidence: 0.9798 - Lift: 1.0041
9: stflife_3_3 -> clsprty_1_3
Support: 0.7620 - Confidence: 0.9797 - Lift: 1.0040
10: stflife_3_3 -> happy_3_3
Support: 0.7378 - Confidence: 0.9485 - Lift: 1.1279
11: happy_3_3, impcntr_1_3 -> clsprty_1_3
Support: 0.7077 - Confidence: 0.9796 - Lift: 1.0039
12: happy_3_3, iplylfr_1_3 -> clsprty_1_3
Support: 0.7183 - Confidence: 0.9834 - Lift: 1.00

In [17]:
concepts = context_small.get_concepts()

In [18]:
show_concepts(concepts, 0.7, 2, num_of_rows)

1: ['clsprty_1_3', 'happy_3_3', 'impcntr_1_3'] - Support : 0.7
2: ['clsprty_1_3', 'health_1_3', 'impcntr_1_3'] - Support : 0.8
3: ['clsprty_1_3', 'health_1_3', 'impcntr_1_3', 'impfree_1_3'] - Support : 0.7
4: ['clsprty_1_3', 'health_1_3', 'impcntr_1_3', 'iplylfr_1_3'] - Support : 0.75
5: ['clsprty_1_3', 'health_1_3', 'impcntr_1_3', 'stfhlth_3_3'] - Support : 0.7
6: ['clsprty_1_3', 'impcntr_1_3'] - Support : 0.9
7: ['clsprty_1_3', 'impcntr_1_3', 'impfree_1_3'] - Support : 0.8
8: ['clsprty_1_3', 'impcntr_1_3', 'ipeqopt_1_3'] - Support : 0.7
9: ['clsprty_1_3', 'impcntr_1_3', 'stfhlth_3_3'] - Support : 0.75
10: ['happy_3_3', 'impcntr_1_3'] - Support : 0.75
11: ['happy_3_3', 'health_1_3', 'impcntr_1_3'] - Support : 0.7
12: ['health_1_3', 'impcntr_1_3'] - Support : 0.85
13: ['health_1_3', 'impcntr_1_3', 'impfree_1_3'] - Support : 0.75
14: ['health_1_3', 'impcntr_1_3', 'ipeqopt_1_3'] - Support : 0.7
15: ['health_1_3', 'impcntr_1_3', 'stfhlth_3_3'] - Support : 0.75
16: ['imdfetn_2_3', 'impcntr

Implications for 9 rounds

In [19]:
get_rules_for_rounds(min_support=0.7, min_confidence=1)


================= Round #1 =================

================= Round #2 =================

================= Round #3 =================

================= Round #4 =================

================= Round #5 =================

================= Round #6 =================

================= Round #7 =================

================= Round #8 =================
1: ipeqopt_1_3 -> clsprty_1_3
Support: 0.7452 - Confidence: 1.0000 - Lift: 1.0006

================= Round #9 =================

==== Rules that appear in 2 or more rounds ====



In [20]:
get_rules_for_rounds(min_support=0.6, min_confidence=1)


================= Round #1 =================

================= Round #2 =================

================= Round #3 =================
1: stfhlth_3_3, freehms_1_3 -> clsprty_1_3
Support: 0.6285 - Confidence: 1.0000 - Lift: 1.0067

================= Round #4 =================

================= Round #5 =================

================= Round #6 =================

================= Round #7 =================

================= Round #8 =================
1: ipeqopt_1_3 -> clsprty_1_3
Support: 0.7452 - Confidence: 1.0000 - Lift: 1.0006
2: ipmodst_1_3 -> clsprty_1_3
Support: 0.6070 - Confidence: 1.0000 - Lift: 1.0006
3: ipudrst_1_3 -> clsprty_1_3
Support: 0.6767 - Confidence: 1.0000 - Lift: 1.0006
4: ipeqopt_1_3, aesfdrk_2_2 -> clsprty_1_3
Support: 0.6206 - Confidence: 1.0000 - Lift: 1.0006
5: ipeqopt_1_3, freehms_1_3 -> clsprty_1_3
Support: 0.6518 - Confidence: 1.0000 - Lift: 1.0006
6: ipeqopt_1_3, happy_3_3 -> clsprty_1_3
Support: 0.6484 - Confidence: 1.0000 - Lift: 1.0006
7: ipeqo

In [21]:
get_rules_for_rounds(min_support=0.5, min_confidence=1)


================= Round #1 =================

================= Round #2 =================

================= Round #3 =================
1: prtdgcl_1_3 -> clsprty_1_3
Support: 0.5350 - Confidence: 1.0000 - Lift: 1.0067
2: stfhlth_3_3, freehms_1_3 -> clsprty_1_3
Support: 0.6285 - Confidence: 1.0000 - Lift: 1.0067
3: stfhlth_3_3, freehms_1_3, aesfdrk_2_2 -> clsprty_1_3
Support: 0.5239 - Confidence: 1.0000 - Lift: 1.0067
4: stfhlth_3_3, happy_3_3, freehms_1_3 -> clsprty_1_3
Support: 0.5567 - Confidence: 1.0000 - Lift: 1.0067
5: stfhlth_3_3, freehms_1_3, health_1_3 -> clsprty_1_3
Support: 0.5061 - Confidence: 1.0000 - Lift: 1.0067
6: stfhlth_3_3, impcntr_1_3, freehms_1_3 -> clsprty_1_3
Support: 0.5551 - Confidence: 1.0000 - Lift: 1.0067
7: ipeqopt_1_3, stfhlth_3_3, freehms_1_3 -> clsprty_1_3
Support: 0.5100 - Confidence: 1.0000 - Lift: 1.0067
8: stfhlth_3_3, freehms_1_3, iplylfr_1_3 -> clsprty_1_3
Support: 0.5684 - Confidence: 1.0000 - Lift: 1.0067
9: stflife_3_3, stfhlth_3_3, freehms_1_3

In [22]:
# get_rules_for_rounds(min_support=0.4, min_confidence=1)

In [23]:
print_top_conf1_rules_by_support(2)  #+- 15 minutes to find


================= Round #1 =================
1: prtdgcl_1_3 -> clsprty_1_3
Support: 0.4081 - Confidence: 1.0000 - Lift: 1.0248
2: prtdgcl_1_3, impcntr_1_3 -> clsprty_1_3
Support: 0.3612 - Confidence: 1.0000 - Lift: 1.0248

================= Round #2 =================
1: prtdgcl_1_3 -> clsprty_1_3
Support: 0.4882 - Confidence: 1.0000 - Lift: 1.0137
2: prtdgcl_1_3, iplylfr_1_3 -> clsprty_1_3
Support: 0.4274 - Confidence: 1.0000 - Lift: 1.0137

================= Round #3 =================
1: stfhlth_3_3, freehms_1_3 -> clsprty_1_3
Support: 0.6285 - Confidence: 1.0000 - Lift: 1.0067
2: stfhlth_3_3, freehms_1_3, iplylfr_1_3 -> clsprty_1_3
Support: 0.5684 - Confidence: 1.0000 - Lift: 1.0067

================= Round #4 =================
1: impcntr_1_3, iplylfr_1_3, iprspot_2_3 -> clsprty_1_3
Support: 0.3847 - Confidence: 1.0000 - Lift: 1.0080
2: prtdgcl_1_3 -> clsprty_1_3
Support: 0.3716 - Confidence: 1.0000 - Lift: 1.0080

================= Round #5 =================
1: impdiff_2_3 -> clspr

In [24]:
from typing import Set


def get_support(df: pd.DataFrame, itemset: Set[str]) -> float:
    """
    Calculate the support of an itemset in the dataframe.

    :param df: The input DataFrame with boolean or binary (0/1 or True/False) attributes.
    :param itemset: A set of attributes/items to compute support for.
    :return: Support value (fraction of rows that contain all items)
    """
    if not itemset:
        return 0.0
    condition = df[list(itemset)].all(axis=1)
    return condition.mean()

In [26]:
def get_confidence(df: pd.DataFrame, base: Set[str], add: Set[str]) -> float:
    """
    Calculate the confidence of a rule: base → add

    :param df: The input DataFrame with binary (1/0 or True/False) data.
    :param base: Set of base (antecedent) items.
    :param add: Set of added (consequent) items.
    :return: Confidence value (P(add | base))
    """
    if not base or not add:
        return 0.0
    support_base = get_support(df, base)
    support_combined = get_support(df, base.union(add))
    if support_base == 0:
        return 0.0
    return support_combined / support_base

In [27]:
def evaluate_itemset_across_rounds(base: Set[str], add: Set[str]) -> None:
    """
    For each round, compute and print:
      - Support of the combined itemset (base ∪ add)
      - Confidence of the rule: base → add

    :param base: Set of base (antecedent) attributes
    :param add: Set of add (consequent) attributes
    :param data_path: Path to folder containing essround{i}.csv files
    """
    if not base or not add:
        print("⚠️ Both base and add must be non-empty sets.")
        return

    full_itemset = base.union(add)

    for round_num in range(1, 10):
        df = pd.read_csv(f"../data/essround{round_num}.csv")

        if not full_itemset.issubset(df.columns):
            print(f"Round {round_num}: ⚠️ Missing attributes in itemset: {full_itemset - set(df.columns)}")
            continue

        support = get_support(df, full_itemset)
        confidence = get_confidence(df, base, add)

        print(f"Round {round_num}:")
        print(f"  Support ({base | add}): {support:.4f}")
        print(f"  Confidence ({base} → {add}): {confidence:.4f}")

In [29]:
base = {"ipeqopt_1_3"}
add = {"clsprty_1_3"}
evaluate_itemset_across_rounds(base, add)

Round 1:
  Support ({'ipeqopt_1_3', 'clsprty_1_3'}): 0.7346
  Confidence ({'ipeqopt_1_3'} → {'clsprty_1_3'}): 0.9776
Round 2:
  Support ({'ipeqopt_1_3', 'clsprty_1_3'}): 0.7745
  Confidence ({'ipeqopt_1_3'} → {'clsprty_1_3'}): 0.9878
Round 3:
  Support ({'ipeqopt_1_3', 'clsprty_1_3'}): 0.7848
  Confidence ({'ipeqopt_1_3'} → {'clsprty_1_3'}): 0.9937
Round 4:
  Support ({'ipeqopt_1_3', 'clsprty_1_3'}): 0.7568
  Confidence ({'ipeqopt_1_3'} → {'clsprty_1_3'}): 0.9903
Round 5:
  Support ({'ipeqopt_1_3', 'clsprty_1_3'}): 0.7694
  Confidence ({'ipeqopt_1_3'} → {'clsprty_1_3'}): 0.9939
Round 6:
  Support ({'ipeqopt_1_3', 'clsprty_1_3'}): 0.7838
  Confidence ({'ipeqopt_1_3'} → {'clsprty_1_3'}): 0.9973
Round 7:
  Support ({'ipeqopt_1_3', 'clsprty_1_3'}): 0.7609
  Confidence ({'ipeqopt_1_3'} → {'clsprty_1_3'}): 0.9963
Round 8:
  Support ({'ipeqopt_1_3', 'clsprty_1_3'}): 0.7452
  Confidence ({'ipeqopt_1_3'} → {'clsprty_1_3'}): 1.0000
Round 9:
  Support ({'ipeqopt_1_3', 'clsprty_1_3'}): 0.7408
  Co

In [30]:
base = {"ipeqopt_1_3", "stflife_3_3", "impcntr_1_3", "iplylfr_1_3", "happy_3_3"}
add = {"clsprty_1_3"}
evaluate_itemset_across_rounds(base, add)

Round 1:
  Support ({'ipeqopt_1_3', 'stflife_3_3', 'happy_3_3', 'impcntr_1_3', 'iplylfr_1_3', 'clsprty_1_3'}): 0.4286
  Confidence ({'ipeqopt_1_3', 'stflife_3_3', 'happy_3_3', 'impcntr_1_3', 'iplylfr_1_3'} → {'clsprty_1_3'}): 0.9855
Round 2:
  Support ({'ipeqopt_1_3', 'stflife_3_3', 'happy_3_3', 'impcntr_1_3', 'iplylfr_1_3', 'clsprty_1_3'}): 0.4370
  Confidence ({'ipeqopt_1_3', 'stflife_3_3', 'happy_3_3', 'impcntr_1_3', 'iplylfr_1_3'} → {'clsprty_1_3'}): 0.9911
Round 3:
  Support ({'ipeqopt_1_3', 'stflife_3_3', 'happy_3_3', 'impcntr_1_3', 'iplylfr_1_3', 'clsprty_1_3'}): 0.4828
  Confidence ({'ipeqopt_1_3', 'stflife_3_3', 'happy_3_3', 'impcntr_1_3', 'iplylfr_1_3'} → {'clsprty_1_3'}): 0.9954
Round 4:
  Support ({'ipeqopt_1_3', 'stflife_3_3', 'happy_3_3', 'impcntr_1_3', 'iplylfr_1_3', 'clsprty_1_3'}): 0.4722
  Confidence ({'ipeqopt_1_3', 'stflife_3_3', 'happy_3_3', 'impcntr_1_3', 'iplylfr_1_3'} → {'clsprty_1_3'}): 0.9916
Round 5:
  Support ({'ipeqopt_1_3', 'stflife_3_3', 'happy_3_3', 'imp

Work with output(change to better names of the columns)

Improve local evaluation